In [1]:
# My analysis and approach
# 1. Read in all the files into a single dataframe
# 2. Use Panda logic construct to identify the rows which satisifies the condition
# 3. Use group by method and determine the count by weather station (i.e., filename)
# 4. Add the files which are missing from the above result 
#       [The program should create one line of output for each of the weather data files]
# 5. write to CSV file with tab separated in ascending order of the filename

In [2]:
# Import necessary packages
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [3]:
# Parametrizing src and destination path
src_path = '/Users/arunachalamev/Downloads/DataSciTest/wx_data/'
des_path = '/Users/arunachalamev/Downloads/DataSciTest/answers/'

# Read the file names in the src path 
wx_data_files = [filename for filename in listdir(src_path) if isfile(join(src_path, filename))]
print (len(wx_data_files))

167


In [4]:
# load all the files into the data frame. NO HEADER given in the input files
dfs = list()
for i, file in enumerate(wx_data_files):
    data = pd.read_csv(src_path+file, sep='\t', header=None)
    data['file_name'] = file
    dfs.append(data)

full_wx_data = pd.concat(dfs,axis=0)

print ("Shape of the dataframe :",full_wx_data.shape)

Shape of the dataframe : (1729957, 5)


In [5]:
# rename the columns for readability
full_wx_data = full_wx_data.reset_index(drop=True)
full_wx_data = full_wx_data.rename(columns={0: "date", 1: "max_temp", 2:"min_temp", 3:"precipitation"})
#full_wx_data.head(10)
#full_wx_data.tail(10)
#full_wx_data.isna().sum() #make sure no na/null

In [6]:
# Question 1 requriments
MISSING_VALUE = -9999

pos_records = full_wx_data[(full_wx_data['max_temp'] != MISSING_VALUE) & 
                        (full_wx_data['min_temp'] != MISSING_VALUE) & 
                        (full_wx_data['precipitation'] == MISSING_VALUE)]

print ("Number of records which meets the criteria:", pos_records.shape)

Number of records which meets the criteria: (22691, 5)


In [7]:
pos_records_grp_by_cnts = pos_records.groupby(by='file_name').size()
pos_records_grp_by_cnts_dict = pos_records_grp_by_cnts.to_dict()

In [8]:
# logic to find missing weatherstation(filename) without any successful records
# Ask: "The program should create one line of output for each of the weather data files"
# Expectation is to have 167 lines in the output file - 
# Ascending requirement : Achieved using sorted filenames in the iterable list

temp = []
for file in sorted(wx_data_files):
    if file in pos_records_grp_by_cnts_dict:
        temp.append([file,pos_records_grp_by_cnts_dict[file]])
    else:
        temp.append([file,0])

In [9]:
answer1 = pd.DataFrame(temp)

In [10]:
answer1

,0,1
0,USC00110072.txt,121
1,USC00110187.txt,6
2,USC00110338.txt,22
3,USC00111280.txt,92
4,USC00111436.txt,125
...,...,...
162,USC00338552.txt,8
163,USC00338769.txt,7
164,USC00338822.txt,9
165,USC00338830.txt,201


In [11]:
answer1.to_csv(des_path+'MisssingPrcpData.out',sep='\t', header=None, index= False)